In [2]:
import os
import sys
import time

import jax
import jax.numpy as jnp
import pytest
from sklearn.exceptions import NotFittedError
from sklearn.datasets import fetch_openml, load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.metrics import root_mean_squared_error

from bde.bde import BdeClassifier, BdeRegressor
from bde.bde_builder import BdeBuilder
from bde.models import Fnn
from bde.task import TaskType

In [ ]:
from pathlib import Path
from sklearn.linear_model import LinearRegression

def test_sanity_airfoil():
    reg = BdeRegressor(
        hidden_layers=[4],
        n_members=3,
        epochs=5,
        warmup_steps=100,
        n_samples=10,
        n_thinning=2,
        patience=2,
    )
    data_path = Path.cwd() / "bde" / "data" / "airfoil.csv"
    data = pd.read_csv(data_path)

    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    x_scaler = StandardScaler()
    y_scaler = StandardScaler()

    X_train = x_scaler.fit_transform(X_train)
    X_test  = x_scaler.transform(X_test)

    y_train = y_scaler.fit_transform(y_train)
    y_test  = y_scaler.transform(y_test)

    # ---- BDE ----
    reg.fit(jnp.array(X_train), jnp.array(y_train))
    bde_mean, _ = reg.predict(jnp.array(X_test), mean_and_std=True)

    bde_rmse = root_mean_squared_error(y_test, bde_mean)
    print(bde_rmse)
    # sanity bound — only catches broken models
    assert bde_rmse < 5.0

    # ---- Linear baseline (optional diagnostic) ----
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr_pred = lr.predict(X_test)
    lr_rmse = root_mean_squared_error(y_test, lr_pred)
    print(lr_rmse)

    # optional: BDE should not be astronomically worse
    assert bde_rmse < lr_rmse * 2

In [86]:
print(test_sanity_airfoil())

c:\Users\Admin\anaconda3\envs\bde\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
MCLMC warmup: 100%|██████████| 100/100 [00:04<00:00, 23.01it/s]
c:\Users\Admin\anaconda3\envs\bde\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


0.7852412704584999
0.6867814222911596
None


In [ ]:
from pathlib import Path
from sklearn.linear_model import LogisticRegression

def test_sanity_iris():
    clf = BdeClassifier(
        hidden_layers=[4],
        n_members=3,
        epochs=5,
        warmup_steps=100,
        n_samples=10,
        n_thinning=2,
        patience=2,
    )

    data_path = Path.cwd() / "bde" / "data" / "iris.csv"
    data = pd.read_csv(data_path)

    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values.ravel()   # keep as integers

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Scale only X
    x_scaler = StandardScaler()
    X_train = x_scaler.fit_transform(X_train)
    X_test  = x_scaler.transform(X_test)

    # ---- BDE ----
    clf.fit(jnp.array(X_train), jnp.array(y_train))
    y_pred = clf.predict(jnp.array(X_test))

    acc = (y_pred == y_test).mean()
    print("Accuracy:", acc)

    # sanity bound — just make sure it's not broken
    assert acc > 0.5


    # ---- Linear baseline (optional diagnostic) ----

    clf = LogisticRegression(max_iter=200)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    acc_log = (pred == y_test).mean()
    print(acc_log)

    # optional: BDE should not be astronomically worse


In [100]:
print(test_sanity_iris())

c:\Users\Admin\anaconda3\envs\bde\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
MCLMC warmup: 100%|██████████| 100/100 [00:04<00:00, 22.44it/s]
c:\Users\Admin\anaconda3\envs\bde\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Accuracy: 0.9333333333333333
1.0
None
